# 🧩 09. 서브워드 토크나이저 (Subword Tokenizer): 단어 쪼개기 기술

## 1. 개요
지금까지는 단어를 띄어쓰기 기준으로 나눴습니다. 하지만 세상엔 새로운 단어가 계속 생겨납니다. (예: `ChatGPT`, `JMT`...)
기계가 모르는 단어(OOV)를 만나면 "모름!" 하고 포기해버리는 문제를 해결하기 위해, 단어를 더 작은 **의미 조각(Subword)**으로 쪼갭니다.

> **쉬운 비유**:  
> - `Unfriendly` 라는 단어를 모른다고 칩시다.
> - 하지만 `Un-` (아닌) + `friend` (친구) + `-ly` (하게) 로 쪼개서 보면 뜻을 알 수 있습니다!
> - 이게 바로 **서브워드(Subword)** 방식입니다.

---

## 2. 왜 서브워드인가요?

| 방식 | 예시 | 장점 | 단점 |
|:---:|---|---|---|
| **단어 단위** | `apple` | 의미 파악 쉬움 | 모르는 단어 나오면 끝장 (OOV) |
| **글자 단위** | `a`, `p`, `p`, `l`, `e` | OOV 없음 | 문장이 너무 길어짐, 의미 파악 어려움 |
| **서브워드** | `app` + `##le` | **OOV 해결 + 적당한 길이** | **(현재 가장 많이 쓰는 방식)** |

---

## 3. 실습 데이터 준비 (네이버 영화 리뷰)

한국어 데이터로 실습하기 위해 네이버 영화 리뷰 데이터(NSMC)를 다운로드합니다.

In [ ]:
import urllib.request
import os
import pandas as pd

# 1. 데이터 다운로드
def download_file(url, save_name):
    if not os.path.exists(save_name):
        print(f"{save_name} 다운로드 중...")
        urllib.request.urlretrieve(url, save_name)
    else:
        print(f"{save_name} 이미 존재함")

download_file('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 'ratings_train.txt')

# 2. 데이터 읽기
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df = train_df.dropna(how='any') # 결측치 제거

print(f"리뷰 개수: {len(train_df)}개")
train_df.head() # 상위 5개 확인

In [ ]:
# 학습을 위해 리뷰 내용만 따로 'txt' 파일로 저장합니다.
with open('naver_review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_df['document'].values))

## 4. 구글의 걸작: SentencePiece (SPM)

구글이 만든 서브워드 토크나이저입니다. 한국어 처리에 아주 강합니다.

### 특징
- 공백을 `_` (밑줄) 문자로 바꿔서 처리합니다.
- 예: `안녕 하세요` ➡️ `_안녕`, `_하세요`
- 이렇게 하면 나중에 문장으로 복구하기가 매우 쉽습니다. (`_`를 공백으로 바꾸기만 하면 됨)

In [ ]:
import sentencepiece as spm

# 1. 학습 시키기 (Train)
# --input: 학습할 파일
# --model_prefix: 저장할 이름
# --vocab_size: 단어 집합 크기 (5000개만 쓰겠다)
# --model_type: 알고리즘 (bpe가 유명함)
spm.SentencePieceTrainer.Train(
    '--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999'
)

print("학습 완료! naver.model, naver.vocab 파일이 생성되었습니다.")

In [ ]:
# 2. 로드하기
sp = spm.SentencePieceProcessor()
sp.Load('naver.model')

# 3. 토큰화 테스트
text = "진짜 정말 재밌는 영화였습니다 꿀잼"
tokens = sp.encode_as_pieces(text)
ids = sp.encode_as_ids(text)

print("토큰:", tokens)
print("ID:", ids)
# 결과 예시: [' 진짜', ' 정말', ' 재미', '있는', ...]
# 앞에 붙은 '_' 기호에 주목하세요! (띄어쓰기 정보)

In [ ]:
# 4. 복원 테스트 (Decode)
decoded_text = sp.decode_ids(ids)
print("복원:", decoded_text)

## 5. BERT의 심장: WordPiece Tokenizer

허깅페이스(Hugging Face) 라이브러리를 통해 사용할 수 있습니다.
BERT 모델이 사용하는 방식으로 유명합니다.

### 특징
- 단어 중간에 이어지는 조각에는 `##`을 붙입니다.
- 예: `playing` ➡️ `play`, `##ing` 
- "##ing는 혼자 쓰이는 게 아니라 앞 단어에 붙어있는 거야" 라는 뜻입니다.

In [ ]:
from tokenizers import BertWordPieceTokenizer

# 1. 토크나이저 생성
tokenizer = BertWordPieceTokenizer(lowercase=False)

# 2. 학습
tokenizer.train(
    files=['naver_review.txt'],
    vocab_size=5000,
    min_frequency=5, # 최소 5번은 나와야 단어로 인정
    wordpieces_prefix='##' # 이어지는 조각 표시 기호
)

# 3. 테스트
encoded = tokenizer.encode("진짜 정말 재밌는 영화였습니다 꿀잼")

print("토큰:", encoded.tokens)
# 결과 예시: ['진짜', '정말', '재밌', '##는', ...]
# '##'이 붙은 것은 접미사라는 뜻!

print("ID:", encoded.ids)

## 마무리 요약

| 구분 | SentencePiece (구글) | WordPiece (BERT) |
|:---:|---|---|
| **공백 처리** | `_` (사용자 편의성 좋음) | 별도 처리 없음 |
| **기호** | 앞 `_` | 뒤 `##` |
| **사용처** | 한국어 등 다양한 언어 범용적 | BERT 계열 모델 |

> **결론**: 요즘 딥러닝(LLM)은 대부분 이 **서브워드 토큰화** 방식을 사용합니다.